In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = " ", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.12.6: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

conv = "<HUMAN>: What is a panic attack?\n<ASSISTANT>: Panic attacks come on suddenly."

so when conv.split("<HUMAN>:")[1]
gives : " What is a panic attack?\n<ASSISTANT>: Panic attacks come on suddenly."

now: conv.split("<HUMAN>:")[1].split("<ASSISTANT>:")
gives:
[
  " What is a panic attack?\n",
  " Panic attacks come on suddenly."
]
now---:

[0] → " What is a panic attack?\n"
[1] → " Panic attacks come on suddenly."

And strip removed extra spaces and newlines from both sides.

In [ ]:
import kagglehub
path = kagglehub.dataset_download("thedevastator/mental-health-chatbot-pairs")

Using Colab cache for faster access to the 'mental-health-chatbot-pairs' dataset.


In [ ]:
print(path)

/kaggle/input/mental-health-chatbot-pairs


In [ ]:
import pandas as pd
import os
os.listdir(path)
# print(df.head())
df = pd.read_csv(os.path.join(path, "train.csv"))
print(df.head())

df.to_csv("train.csv", index= False)   #to load the csv

                                                text
0  <HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1  <HUMAN>: What are symptoms of panic attack vs....
2  <HUMAN>: What are the types of Mental Illness?...
3  <HUMAN>: What does mental-illness mean?\n<ASSI...
4  <HUMAN>: How can you treat mental illness?\n<A...


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="train.csv",
    split="train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.column_names

['text']

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    texts= []
    for conv in examples["text"]:
      if "<HUMAN>:" in conv and "<ASSISTANT>:" in conv:
        human = conv.split("<HUMAN>:")[1].split("<ASSISTANT>:")[0].strip()
        assistant = conv.split("<ASSISTANT>:")[1].strip()


        instruction = "You are a supportive and empathetic mental health assistant.Accordingly answer the questions asked"
        input       = human
        output      = assistant

          # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
        return { "text" : texts, }
pass


dataset = dataset.map(formatting_prompts_func, remove_columns=dataset.column_names)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

dataset.map(function) means:

“Run this function on EVERY ROW of the dataset”,


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        num_train_epochs = 1, #for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
8.957 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 172 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,1.400400
2,1.180200
3,1.395600
4,1.190600
5,1.378600
6,0.945500
7,1.158300
8,1.244500
9,1.315800
10,0.847300


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a supportive and empathetic mental health assistant.Accordingly answer the questions asked ", # instruction
        "How to manage stress?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a supportive and empathetic mental health assistant.Accordingly answer the questions asked \n\n### Input:\nHow to manage stress?\n\n### Response:\n1. Take a step back: When you're feeling overwhelmed, it's essential to take a step back and try to gain some perspective. Remember that stress is a normal human reaction to challenging situations, and it's okay to feel stressed from time to time.\n\n2. Practice deep breathing: Deep"]

In [ ]:
model.push_to_hub("Full-Finetuned-Mistral-v3-7b")
tokenizer.push_to_hub("Full-Finetuned-Mistral-v3-7b")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  15%|#4        | 25.1MB /  168MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/Full-Finetuned-Mistral-v3-7b


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...pf9y1vh5r/tokenizer.model: 100%|##########|  587kB /  587kB            

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
model.push_to_hub_merged(
    "prranjal/16-bit-Mistral-7B",
    tokenizer,
    save_method="merged_16bit"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...istral-7B/tokenizer.model: 100%|##########|  587kB /  587kB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0003-of-00003.safetensors:   1%|          | 41.8MB / 4.55GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [07:43<00:00, 154.54s/it]


Unsloth: Merge process complete. Saved to `/content/prranjal/16-bit-Mistral-7B`
